<a href="https://colab.research.google.com/github/MercyMoparthy/lab-chains-in-langchain/blob/main/lab-chains-in-langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
from google.colab import userdata
userdata.get('Langchain')
OpenAI_api_key = userdata.get('Langchain')

In [ ]:
#!pip install pandas

In [14]:
import pandas as pd
import os
print(os.getcwd())
df = pd.read_csv('/content/Data.csv')
print(os.listdir())
if 'data' in os.listdir():
    print(os.listdir('data'))

/content
['.config', 'Data.csv', 'sample_data']


In [15]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [16]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [17]:
%pip install langchain_openai
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.8 MB/s eta 0:00:00


In [18]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7, api_key=OpenAI_api_key)


In [19]:
prompt = ChatPromptTemplate.from_template( 'Provide detailed description with features, availability, price of the {product}'

)

In [20]:

chain = LLMChain(llm=llm, prompt=prompt)

In [21]:
product = "Pillows Insert"
chain.run(product=product)

'Pillow inserts are essential for adding volume and shape to decorative pillow covers. They come in various sizes, shapes, and materials to suit different preferences and needs. \n\nFeatures:\n- Available in different sizes such as standard, queen, king, and Euro sizes to fit standard pillow covers.\n- Made from various materials including down/feather blends, polyester, and memory foam for different levels of firmness and support.\n- Some inserts are hypoallergenic and suitable for individuals with allergies.\n- Zippered covers for easy removal and washing.\n- Some inserts are reversible with different firmness levels on each side.\n- Some inserts are adjustable, allowing you to add or remove filling to customize the firmness level.\n\nAvailability:\nPillow inserts are widely available in home goods stores, department stores, online retailers, and specialty bedding stores. They can also be found in various sizes and shapes to fit different types of decorative pillow covers.\n\nPrice:\

## SimpleSequentialChain

In [22]:
from langchain.chains import SimpleSequentialChain

In [23]:
llm = ChatOpenAI(temperature=0.9, api_key=OpenAI_api_key)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
   'Provide detailed description with features, availability, price of the {product}'
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [24]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    'Provide overview of all the reviews {product}'
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [25]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [26]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Pillows inserts are essential components of throw pillows, providing the necessary structure and support to ensure a comfortable and aesthetically pleasing result. They are typically made from a variety of materials, including polyester, down, feathers, and foam, each offering its own unique benefits.

Features:

1. Material: Pillow inserts come in various materials, each with different qualities. Down and feather inserts are known for their softness and pliability, while polyester inserts are more firm and durable.

2. Size: Pillow inserts come in a range of sizes to fit different pillow covers. Common sizes include 16x16, 18x18, and 20x20 inches.

3. Fill Power: For down and feather inserts, fill power indicates the quality and fluffiness of the fill. The higher the fill power, the more luxurious and durable the insert will be.

4. Hypoallergenic: Some pillow inserts are labeled as hypoallergenic, making them suitable for those with all

'Overall, customers tend to be satisfied with pillow inserts, noting their ability to provide the needed fluffiness and support for their throw pillows. Many appreciate the range of materials and sizes available, allowing them to find the perfect insert for their needs. Customers also value the hypoallergenic options for those with sensitivities. While some find the price of higher-quality inserts to be on the higher side, most agree that the comfort and durability they provide make them worth the investment.Overall,pillow inserts are a recommended purchase for those looking to enhance the comfort and aesthetics of their throw pillows.'

**Repeat the above twice for different products**

## SequentialChain

In [39]:
from langchain.chains import SequentialChain

In [40]:
llm = ChatOpenAI(temperature=0.9, api_key=OpenAI_api_key)


first_prompt = ChatPromptTemplate.from_template(
  'Translate the review to English: {English_Review}'
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key='English_Review'
                    )


In [41]:
second_prompt = ChatPromptTemplate.from_template(
    'Translate the Review into German: {English_Review}'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key='German_Review' #give a name to this output
                    )


In [44]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    'Translate the Review into Hindi Language: {German_Review}'
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='Hindi_Review'
                      )


In [45]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        'Here is the HIndi Translation: {Hindi_Review}\nHere is the German Trnaslation: {German_Review}\nHere is the English Translation: {English_Review}\n Based on these translations, provide a brief follow-up message'
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='followup_message'
                     )


In [46]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['review'],
    output_variables=['English_Review', 'German_Review','Hindi_Review', 'Followup_message'],
    verbose=True
)

ValidationError: 1 validation error for SequentialChain
  Value error, Missing required input keys: {'English_Review'}, only had {'review'} [type=value_error, input_value={'chains': [LLMChain(verb...sage'], 'verbose': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
review = df.Review[5]
overall_chain(review)

**Repeat the above twice for different products or reviews**

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

**Repeat the above at least once for different inputs and chains executions - Be creative!**